# Testing datamodule

## 1. Workspace setup

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [ ]:
import sys
import os
import json
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from spinediffusion.datamodule.datamodule import SpineDataModule

## 2. Instantiate datamodule

In [ ]:
transform_args = {
    # "spine_length_normalize": {
    #     "transform_number": 0,
    #     "norm_length": 1,
    # },
    "resample_3d_curve": {
        "transform_number": 0,
        "n_points": 1024,
    },
    "resample_point_cloud": {
        "transform_number": 1,
        "n_points": 65536,
        "method": "uniform",
    },
    "project_to_plane": {
        "transform_number": 2,
        "height": 124,
        "width": 124,
        "spine_factor": 1,
        "intensity": 1,
        "method": "median",
        "z_lims": [-150, 100],
    },
    "close_depthmap": {"transform_number": 3, "se_size": (3, 3)},
    "tensorize": {"transform_number": 4},
}

exclude_patients = {
    # "balgrist": [1, 5],
    # "croatian": [4, 5],
    # "italian": [],
    # "ukbb": [],
}

datamodule = SpineDataModule(
    data_dir="P:\\Projects\\LMB_4Dspine\\back_scan_database",
    batch_size=32,
    train_fraction=0.8,
    val_fraction=0.1,
    test_fraction=0.1,
    transform_args=transform_args,
    num_subjects=10,
    exclude_patients=exclude_patients,
    use_cache=False,
    cache_dir="../cache/",
    predict_size=16,
    num_workers=1,
    conditional=True,
)
datamodule.setup(stage=None)

In [ ]:
num_samples = 3
sample_keys = np.random.choice(list(datamodule.data.keys()), num_samples, replace=False)

fig, axs = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))

for i, key in enumerate(sample_keys):
    sample = datamodule.data[key]

    axs[i, 0].imshow(sample["depth_map"].squeeze(0), cmap="gray")
    axs[i, 0].set_title("Back scan depth map")
    axs[i, 1].imshow(sample["esl_depth_map"], cmap="gray")
    axs[i, 1].set_title("ESL depth map")
    axs[i, 2].imshow(sample["isl_depth_map"], cmap="gray")
    axs[i, 2].set_title("ISL depth map")

plt.show()

In [ ]:
# Same as above but with the esl and isl superimposed on the back scan with a different color make it a N x 4 subplot

num_samples = 16
sample_keys = np.random.choice(list(datamodule.data.keys()), num_samples, replace=False)

n_cols = 4
n_rows = int(np.ceil(num_samples / n_cols))

fig, axs = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))

for i, key in enumerate(sample_keys):
    sample = datamodule.data[key]

    back_scan = sample["depth_map"].squeeze(0)
    esl = sample["esl_depth_map"]
    isl = sample["isl_depth_map"]

    back_scan = np.stack([back_scan, back_scan, back_scan], axis=-1)
    back_scan = back_scan / back_scan.max()

    esl = np.stack([esl, np.zeros_like(esl), np.zeros_like(esl)], axis=-1)
    isl = np.stack([np.zeros_like(isl), isl, np.zeros_like(isl)], axis=-1)

    superimposed = back_scan + esl + isl
    superimposed = superimposed / superimposed.max()

    axs.flat[i].imshow(superimposed)
    axs.flat[i].set_title(key)

    # add legend and remove axes
    axs.flat[i].axis("off")

    # Add the legend only once at the top right corner of the figure
    if i == 3:
        axs.flat[i].text(125, 10, "ESL", color="red", fontsize=24)
        axs.flat[i].text(125, 30, "ISL", color="green", fontsize=24)

plt.show()

In [ ]:
from scipy.interpolate import splprep, splev


def parameterize_curve(points, s=0):
    tck, u = splprep(points.T, s=s)
    return tck, u


tcks = [parameterize_curve(sample["isl"])[0] for sample in datamodule.data.values()]

t = np.stack([tck[0] for tck in tcks])
c = np.stack([tck[1] for tck in tcks])
k = [tck[2] for tck in tcks]

fig, axs = plt.subplots(1, 5, figsize=(15, 5))

axs[0].hist(np.concatenate(t), bins=100)
axs[0].set_title("t values")
axs[1].hist(np.concatenate(c), bins=100)
axs[1].set_title("c values")
axs[2].hist(np.array(k), bins=100)
axs[2].set_title("k values")
axs[3].hist(np.array([len(t_i) for t_i in t]), bins=100)
axs[3].set_title("n t values")
axs[4].hist(np.array([len(c_i) for c_i in c]), bins=100)
axs[4].set_title("n c values")

plt.show()

In [ ]:
t_mean = np.mean(t, axis=0)
t_cov = np.cov(t.reshape(-1, t.shape[-1]).T)

c_mean = np.mean(c, axis=0)
c_cov = np.cov(c.reshape(-1, c.shape[-1]).T)

t_dist = np.random.multivariate_normal(t_mean, t_cov)
c_dist = np.random.multivariate_normal(c_mean, c_cov)

In [ ]:
c = np.stack(c).reshape()
c.shape

In [ ]:
means = {}
stds = {}

means_diff = []
stds_diff = []

for key in data.keys():
    backscan = data[key]["backscan"]
    backscan_upsampled = data_upsampled[key]["backscan"]

    means[key] = {}
    stds[key] = {}

    means[key]["original"] = np.mean(np.asarray(backscan.points), axis=0)
    stds[key]["original"] = np.std(np.asarray(backscan.points), axis=0)
    means[key]["upsampled"] = np.mean(np.asarray(backscan_upsampled.points), axis=0)
    stds[key]["upsampled"] = np.std(np.asarray(backscan_upsampled.points), axis=0)

In [ ]:
x_mean_diff = []
y_mean_diff = []
z_mean_diff = []

x_std_diff = []
y_std_diff = []
z_std_diff = []

for key in means.keys():
    x_mean_diff.append(means[key]["original"][0] - means[key]["upsampled"][0])
    y_mean_diff.append(means[key]["original"][1] - means[key]["upsampled"][1])
    z_mean_diff.append(means[key]["original"][2] - means[key]["upsampled"][2])

    x_std_diff.append(stds[key]["original"][0] - stds[key]["upsampled"][0])
    y_std_diff.append(stds[key]["original"][1] - stds[key]["upsampled"][1])
    z_std_diff.append(stds[key]["original"][2] - stds[key]["upsampled"][2])

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

axs[0, 0].hist(x_mean_diff, bins=20)
axs[0, 0].set_title("X mean difference")
axs[0, 0].set_xlabel("Difference")
axs[0, 0].set_ylabel("Frequency")

axs[0, 1].hist(y_mean_diff, bins=20)
axs[0, 1].set_title("Y mean difference")
axs[0, 1].set_xlabel("Difference")
axs[0, 1].set_ylabel("Frequency")

axs[0, 2].hist(z_mean_diff, bins=20)
axs[0, 2].set_title("Z mean difference")
axs[0, 2].set_xlabel("Difference")
axs[0, 2].set_ylabel("Frequency")

axs[1, 0].hist(x_std_diff, bins=20)
axs[1, 0].set_title("X std difference")
axs[1, 0].set_xlabel("Difference")
axs[1, 0].set_ylabel("Frequency")

axs[1, 1].hist(y_std_diff, bins=20)
axs[1, 1].set_title("Y std difference")
axs[1, 1].set_xlabel("Difference")
axs[1, 1].set_ylabel("Frequency")

axs[1, 2].hist(z_std_diff, bins=20)
axs[1, 2].set_title("Z std difference")
axs[1, 2].set_xlabel("Difference")
axs[1, 2].set_ylabel("Frequency")

In [ ]:
height = 128  # pixels

factor = 1

areas = []

for key in data.keys():
    special_points = data[key]["special_points"]
    C7 = special_points["C7"]
    DR = special_points["DR"]
    DL = special_points["DL"]

    DM = (DR + DL) / 2

    spine_length = np.linalg.norm(C7 - DM)  # length units

    spine_pixels = height * factor

    d_height = spine_length / spine_pixels

    areas.append(d_height**2)


plt.hist(areas, bins=20)
plt.title("Pixel area")
plt.xlabel("Area [$mm^2$]")
plt.ylabel("# subjects")

plt.show()

In [ ]:
indices = np.random.choice(len(datamodule.data.keys()), 16, replace=False)

fig, axs = plt.subplots(4, 4, figsize=(15, 15))

for i, idx in enumerate(indices):
    key = list(datamodule.data.keys())[idx]

    depth_map = datamodule.data[key]["depth_map"]
    axs.flat[i].imshow(depth_map[0], cmap="gray")